## Импорт зависимостей

In [ ]:
import os
import random
import shutil
import json
from shapely.geometry import Polygon

# Загрузка 'Traffic Signs' dataset

In [ ]:
!wget -q 'https://www.dropbox.com/scl/fi/aptgg7do2iruqeyo9exrn/traffic-signs.tar.gz?rlkey=l6dl8nel7a8wk36p7dlns8kx8&dl=1' -O 'traffic-signs.tar.gz'

In [ ]:
!tar -zxf 'traffic-signs.tar.gz'

# ТЗ

## Road Sign Annotation Project

## Project Overview
This project involves annotating a dataset of 50 images that depict various road scenes, specifically focusing on road signs. The task requires identifying and classifying each road sign into one of four distinct classes: stop, crosswalk, traffic light, and speed limit. The dataset will be uploaded and annotated using the [Supervisely](https://app.supervisely.com) platform.

## Objective
- To accurately annotate road signs in each image.
- Classify each sign into one of the specified categories.

## Dataset Details
- **Total Images**: 50
- **Content Description**: Each image contains one or more road signs in various road scenes.
- **Annotation Platform**: [Supervisely](https://app.supervisely.com)

## Annotation Guidelines
1. **Classes for Annotation**:
   - **stop**: STOP signs.
   - **crosswalk**: Signs indicating crosswalks.
   - **trafficlight**: Traffic lights.
   - **speedlimit**: Signs indicating speed limits on the road.
2. **Annotation Process**:
   - Thoroughly examine each image for road signs.
   - Use the rectangle tool in [Supervisely](https://app.supervisely.com) to draw bounding boxes around each sign.
   - Ensure the bounding box fits closely around the edges of the sign.
3. **Handling Multiple Objects**:
   - If multiple signs are present in an image, annotate each sign separately.
   - Each eligible sign must be properly assigned to one of the four classes.
4. **Accuracy**:
   - Ensure high accuracy in both the placement of bounding boxes and the classification of signs.
   - Recheck annotations for any possible errors.

## Exporting Annotations
- Upon completion, export the annotations in JSON format.
- The JSON file should accurately reflect all annotations and classifications.

## Deliverables
- A JSON file containing the annotations for all 50 images.

## Examples:
![1](https://i.ibb.co/YZD7bGp/1.png)
![2](https://i.ibb.co/Yd9tzfn/2.png)

# Формирование семпла для разметчика, а так же валидационного семпла

In [ ]:
def create_sample(source, target, size, seed=42):
    random.seed(seed)

    dataset_dir = source
    target_dir = target

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    all_images = os.listdir(dataset_dir)
    selected_images = random.sample(all_images, size)

    for image in selected_images:
        source_path = os.path.join(dataset_dir, image)
        target_path = os.path.join(target_dir, image)
        shutil.copy(source_path, target_path)

In [ ]:
create_sample(source='traffic-signs/images', target='hw2-dataset/images', size=50)

In [ ]:
!tar -czf 'hw2-traffic-signs.tar.gz' 'hw2-dataset/images'

In [ ]:
create_sample(source='hw2-dataset/images', target='hw2-dataset/validation-sample', size=10)

# Разметка

Проведена разметка валидационного семпла, подмешанного в основной семпл разметчика

In [ ]:
!mkdir 'results'
%cd results

In [ ]:
!wget -q 'https://www.dropbox.com/scl/fi/vc3dyn797fb9r3o088ky5/marked-up-validation-signs.tar.gz?rlkey=jsni9rlmg40lc3jxu2upt200g&dl=1' -O 'marked-up-validation-signs.tar.gz'
!tar -zxf 'marked-up-validation-signs.tar.gz'

Передан основной семпл, получена аннотации знаков по 4 классам от разметчика

In [ ]:
!wget -q 'https://www.dropbox.com/scl/fi/lggjo73rl2gkb73u91q3k/marked-up-signs.tar.gz?rlkey=e2oycef22ac34j1up7qfpzpdw&dl=1' -O 'marked-up-signs.tar.gz'
!tar -zxf 'marked-up-signs.tar.gz'

# Валидация

Количество аннотаций соответствует количеству объектов в семпле

In [ ]:
!ls 'annotations/' | wc -l

      50


Сверка с валидационным семплом, на количество объектов, типы классов, а так же IoU

In [ ]:
def calculate_iou(boxA, boxB):
    polyA = Polygon([boxA[0], [boxA[1][0], boxA[0][1]], boxA[1], [boxA[0][0], boxA[1][1]]])
    polyB = Polygon([boxB[0], [boxB[1][0], boxB[0][1]], boxB[1], [boxB[0][0], boxB[1][1]]])
    intersection = polyA.intersection(polyB).area
    union = polyA.union(polyB).area
    return intersection / union if union != 0 else 0


def load_annotations(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data["objects"]


def compare_annotations(test_annotations, marked_up_annotations):
    if len(test_annotations) != len(marked_up_annotations):
        print("The number of objects does not match between the annotations\n")
        return

    matched_reviewer_objects = set()

    for idx, test_obj in enumerate(test_annotations, 1):
        test_class = test_obj['classTitle']
        test_box = test_obj['points']['exterior']
        best_iou_score = 0
        best_match_id = None

        for reviewer_obj in marked_up_annotations:
            if reviewer_obj['classTitle'] == test_class and reviewer_obj['id'] not in matched_reviewer_objects:
                reviewer_box = reviewer_obj['points']['exterior']
                iou_score = calculate_iou(test_box, reviewer_box)
                if iou_score > best_iou_score:
                    best_iou_score = iou_score
                    best_match_id = reviewer_obj['id']

        if best_match_id is not None:
            matched_reviewer_objects.add(best_match_id)
            print(f"Object {idx}: Best match found for class '{test_class}'. IoU Score: {round(best_iou_score, 2)}\n")
        else:
            print(f"Class '{test_class}' not found among the reviewer's annotations\n")

In [ ]:
test_annotation_dir = 'validation-annotations/'
marked_up_annotation_dir = 'annotations/'
test_annotations = os.listdir(test_annotation_dir)

for ann in sorted(test_annotations):
    print(f'--- "{ann}" annotation ---')
    test_annotations = load_annotations(test_annotation_dir + ann)
    marked_up_annotations = load_annotations(marked_up_annotation_dir + ann)
    compare_annotations(test_annotations, marked_up_annotations)

--- "road162.json" annotation ---
Object 1: Best match found for class 'speedlimit'. IoU Score: 0.92

--- "road176.json" annotation ---
Object 1: Best match found for class 'trafficlight'. IoU Score: 0.92

Object 2: Best match found for class 'crosswalk'. IoU Score: 0.94

--- "road240.json" annotation ---
Object 1: Best match found for class 'speedlimit'. IoU Score: 0.94

Object 2: Best match found for class 'speedlimit'. IoU Score: 0.83

--- "road274.json" annotation ---
Object 1: Best match found for class 'speedlimit'. IoU Score: 0.9

--- "road459.json" annotation ---
Object 1: Best match found for class 'crosswalk'. IoU Score: 0.88

Object 2: Best match found for class 'trafficlight'. IoU Score: 0.94

Object 3: Best match found for class 'speedlimit'. IoU Score: 0.94

--- "road502.json" annotation ---
Object 1: Best match found for class 'speedlimit'. IoU Score: 0.96

--- "road512.json" annotation ---
Object 1: Best match found for class 'speedlimit'. IoU Score: 0.89

--- "road611.

# Итоги

В процессе выполнении работы был выделен семпл для разметки, в котором содержались подмешанные данные (20%) для валидации. Этот семпл формировался случайным образом, чтобы минимизировать шанс попадания в узкий диапазон, в итоге имеем довольно хорошее распределение валидационного семпла по всему основному.

После получение от разметчика аннотаций при помощи скрипта была произведена оценка приближенности к валидационному семплу при помощи IoU. Как можно видеть, количество объектов и их классы совпадают с валидационном семплом, а IoU достаточно высоко (мин 0.82).

Таким образом, можно сделать вывод, что разметчик произвел разметку довольно качественно и имеем пригодные для дальнейшей работы аннотации.